# Simulador de colas en cine

### Complementos necesarios

In [40]:
pip install simpy

Note: you may need to restart the kernel to use updated packages.


### Librerias

In [41]:
import simpy
import random
import pandas as pd

#### Creacion de clientes

In [42]:

class Persona:
    ultimo_id = 0
        
    def __init__(self):
        Persona.ultimo_id += 1
        self.id = Persona.ultimo_id
        self.edad = random.randint(14, 60)
        self.genero = random.choice(["Femenino", "Masculino"])
        self.nivel_paciencia = random.randint(1, 3)
        
    def tiempo_decidir(self):
        return self.edad * 0.02 + self.nivel_paciencia

#### Definir numero de personas

In [43]:
num_personas = 40

#### Generacion de n clientes

In [44]:

personas = [Persona() for _ in range(num_personas)]

# Crear DataFrame
df = pd.DataFrame(
    data={
        "Identificacion": [persona.id for persona in personas],
        "Edad": [persona.edad for persona in personas],
        "Genero": [persona.genero for persona in personas],
        "Nivel_Paciencia": [persona.nivel_paciencia for persona in personas],
        "Tiempo_Decidir (min)": [persona.tiempo_decidir() for persona in personas]
    }
)

In [45]:
df

,Identificacion,Edad,Genero,Nivel_Paciencia,Tiempo_Decidir (min)
0,1,37,Masculino,1,1.74
1,2,17,Femenino,2,2.34
2,3,55,Masculino,2,3.10
3,4,30,Masculino,1,1.60
4,5,44,Masculino,1,1.88
5,6,40,Masculino,2,2.80
6,7,45,Femenino,1,1.90
7,8,49,Masculino,3,3.98
8,9,21,Femenino,3,3.42
9,10,15,Masculino,1,1.30


#### Creación de empleados

In [46]:
import random

class Empleado:
    ultimo_id = 0
        
    def __init__(self):
        Empleado.ultimo_id += 1
        self.id = Empleado.ultimo_id
        self.experiencia = random.randint(50, 100) / 100  # Genera un número entre 50 y 100 y lo convierte en decimal
        #donde 100:0 experiencia, 99:1 de experiencia, y seguido hasta 50:50 de experiencia.
        self.tiempo_general = 5
        
    def tiempo_promedio(self):
        return self.tiempo_general * self.experiencia


#### Definir numero de servidores

In [47]:
num_servidores = 3

#### Generación de empleados en base a los servidores

In [48]:
empleados = [Empleado() for _ in range(num_servidores)]

In [49]:
df_empleados = pd.DataFrame(
    data={
        "Identificacion": [empleado.id for empleado in empleados],
        "Experiencia": [empleado.experiencia for empleado in empleados],
        "Tiempo general de Empleado (min)": [empleado.tiempo_general for empleado in empleados],
        "Tiempo promedio de Empleado (min)": [empleado.tiempo_promedio() for empleado in empleados]
    }
)

In [50]:
df_empleados

,Identificacion,Experiencia,Tiempo general de Empleado (min),Tiempo promedio de Empleado (min)
0,1,0.88,5,4.40
1,2,0.67,5,3.35
2,3,0.55,5,2.75


# No chambeo mas, porque ya me dio pereza ~cmmz 


**Extra:Posible muestra de asientos usados en la sala**

In [51]:
def crear_matriz():
    filas = ['A', 'B', 'C', 'D', 'E','F','G']
    columnas = [1, 2, 3, 4, 5, 6, 7]

    matriz = []

    for fila in filas:
        nueva_fila = []
        for columna in columnas:
            # La disponibilidad se establece inicialmente en True para todas las celdas
            celda = {'fila': fila, 'columna': columna, 'disponibilidad': True}
            nueva_fila.append(celda)
        matriz.append(nueva_fila)

    return matriz

def imprimir_matriz(matriz):
     for fila in matriz:
        for celda in fila:
            # Seleccionar el código de color basado en la disponibilidad
            color = '\033[92m' if celda['disponibilidad'] else '\033[91m'
            
            # Imprimir la celda en formato 'FilaColumna' con el color correspondiente
            print(f"{color}{celda['fila']}{celda['columna']}\033[0m", end='\t')
        print()  # Saltar a una nueva línea después de imprimir una fila completa

# Crear la matriz
mi_matriz = crear_matriz()

# Imprimir la matriz
imprimir_matriz(mi_matriz)

A1	A2	A3	A4	A5	A6	A7	
B1	B2	B3	B4	B5	B6	B7	
C1	C2	C3	C4	C5	C6	C7	
D1	D2	D3	D4	D5	D6	D7	
E1	E2	E3	E4	E5	E6	E7	
F1	F2	F3	F4	F5	F6	F7	
G1	G2	G3	G4	G5	G6	G7	
